<a href="https://colab.research.google.com/github/danielavilam/colab-research-google/blob/main/C7_RAG_DanielAvila_AIENGINEER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LABORATORIO RAG - DANIEL AVILA**

## **1. Indexing (Preparación de datos)**

### Instalación de librerías

In [1]:
# chromadb → Es la base de datos vectorial donde guardaremos nuestros documentos transformados en vectores
# pandas → Sirve para manejar datos en tablas y facilitar análisis

!pip install chromadb
!pip install pandas

In [2]:
#  Importación de librerías
from google import genai
from google.genai import types
from google.colab import userdata #from dotenv import load_dotenv
from google.api_core import retry

# Utilidades adicionales
import pandas as pd
import os
import re

# Librerías de ChromaDB
from chromadb import Documents, EmbeddingFunction, Embeddings
from chromadb import Client

# Visualización en Colab
from IPython.display import Markdown

### **Configuración de Gemini y modelos embedding**

En la barra lateral izquierda (ícono de la llave), inserta tu API Key con el nombre GEMINI_API_KEY, previamente generado en https://aistudio.google.com/

In [3]:
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

Con la variable client se crea la conexión con Gemini de Google.
Luego, usando este cliente, puedes listar los modelos disponibles.
El filtro embedContent permite mostrar únicamente los modelos que soportan la creación de embeddings.

In [4]:
client = genai.Client(api_key=GEMINI_API_KEY)

for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)


models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp
models/gemini-embedding-001


### **Definición de documentos de prueba**

En este paso definimos textos de ejemplo que representarán documentos dentro de nuestra base de datos vectorial.
Estos servirán para realizar búsquedas y probar cómo el modelo recupera y utiliza la información.

En un entorno real, estos documentos podrían ser reemplazados por información proveniente de archivos PDF, JSON, páginas web o datos empresariales.

In [5]:
# Documentos de ejemplo

# Lista de documentos que se almacenarán en la base vectorial
doc_1 = """En el 2015, el 21,77% de la población peruana vivía en situación de pobreza monetaria, lo que equivale a aproximadamente 6.782.000 personas, según el Instituto Nacional de Estadística e Informática (INEI). Este porcentaje representó una disminución de 1 punto porcentual respecto al año anterior, haciendo que 221 mil peruanos dejaran de ser pobres. La reducción de la pobreza se observó más en las regiones de Selva y Sierra, especialmente en la Selva urbana y Sierra rural."""

doc_2 = """En el año 2020, la pobreza monetaria en Perú afectó al 30.1% de la población, lo que representó un incremento significativo de 9.9 puntos porcentuales respecto al año 2019. Esta cifra marcó un pico de pobreza, siendo el año 2020 el que alcanzó el nivel más alto desde el año 2010, debido a la fuerte crisis sanitaria provocada por la pandemia de COVID-19."""

doc_3 = """En el Perú, la pobreza monetaria en 2024 afectó al 27.6% de la población, lo que equivale a 9.395 millones de personas, según el INEI. Esta cifra representa una disminución de 1.4 puntos porcentuales respecto a 2023, mostrando una leve mejora, aunque los niveles aún están lejos de los registrados antes de la pandemia. La pobreza extrema, por su parte, se ubicó en 5.5% en 2024, también con una leve reducción."""

doc_4 = """Toda la informacion brindada solo se aplica para el Peru"""

documents = [doc_1, doc_2, doc_3, doc_4]

### **Clase de Embedding**

Esta clase de Python encapsula la generación de embeddings usando Gemini.
Trabaja en dos modos:

- document_mode = True → genera embeddings para documentos (fase de Indexing).
- document_mode = False → genera embeddings para consultas (fase de Retrieval).

El método __call__ detecta el modo y solicita a Gemini el vector adecuado (retrieval_document o retrieval_query).
Devuelve una lista de vectores (uno por elemento de entrada) lista para guardarse en la base vectorial o para hacer búsquedas.

In [6]:
class EmbeddingGenerator(EmbeddingFunction):
    # Modo de operación por defecto:
    # True  -> generar embeddings para DOCUMENTOS (Indexing)
    # False -> generar embeddings para CONSULTAS (Retrieval)
    document_mode = True

    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        # Llamada al endpoint de embeddings de Gemini
        # - model:          familia de modelos de embeddings
        # - contents:       lista de textos a vectorizar (Documents o Queries)
        # - task_type:      le indica al modelo el contexto del embedding (document/query)

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )

        # Normaliza la salida: extrae los vectores (lista de floats) por cada item
        return [e.values for e in response.embeddings]

### **Crear la Base Vectorial en ChromaDB**

En esta etapa inicializamos una base de datos vectorial utilizando ChromaDB.
Aquí insertamos los documentos previamente definidos, los cuales se transforman en vectores mediante la clase EmbeddingGenerator.

De esta forma, los documentos quedan listos para ser buscados de manera semántica y eficiente durante la fase de Retrieval.

In [7]:
# Nombre de la colección dentro de ChromaDB
DB_NAME = "testdb"

# Creamos una instancia de la función de embeddings
embed_fn = EmbeddingGenerator()

# Modo documento -> genera embeddings para INDEXING
embed_fn.document_mode = True

# Inicializamos el cliente de ChromaDB
chroma_client = Client()

# Creamos una colección en la base vectorial
vector_db  = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

# Agregamos los documentos a la colección
vector_db.add(documents=documents,
       ids=[str(i) for i in range(len(documents))]
    )

/tmp/ipython-input-91028937.py:5: DeprecationWarning: The class EmbeddingGenerator does not implement __init__. This will be required in a future version.
  embed_fn = EmbeddingGenerator()


In [8]:
# Verificamos que se hayan insertado correctamente
vector_db.count()   # Devuelve el número de documentos almacenados

4

## **2. Retrieval (Recuperación de información)**

En esta etapa pasamos de trabajar con documentos a trabajar con consultas del usuario.
Para lograrlo, cambiamos el modo de embeddings a False, lo que indica que ahora el sistema debe generar un vector a partir de la pregunta en lugar de un documento.

Ese vector de la consulta se compara contra la base de datos vectorial (ChromaDB) y devuelve los fragmentos más relevantes según su similitud semántica.

In [9]:
# Cambiamos a modo "consulta" para generar embeddings de la QUERY del usuario
embed_fn.document_mode = False

# Definimos la pregunta del usuario
#user_query = "En cuánto aumentó el salario mínimo vital en Perú, en 2025, con respecto al 2024"
user_query = "En qué año se registra el menor índice de pobreza en Perú?"

# Realizamos la búsqueda en la base vectorial
# - query_texts: la consulta en formato texto
# - n_results: número de fragmentos más similares que queremos recuperar (Top-K)
search_results  = vector_db.query(query_texts=[user_query], n_results=3)

# Extraemos los documentos recuperados
[retrieved_passages] = search_results ["documents"]

# Mostramos el primer fragmento recuperado en formato Markdown
Markdown(retrieved_passages [0])

En el año 2020, la pobreza monetaria en Perú afectó al 30.1% de la población, lo que representó un incremento significativo de 9.9 puntos porcentuales respecto al año 2019. Esta cifra marcó un pico de pobreza, siendo el año 2020 el que alcanzó el nivel más alto desde el año 2010, debido a la fuerte crisis sanitaria provocada por la pandemia de COVID-19.

## **3. Augmented Generation (Generación enriquecida)**

En esta fase generamos el prompt final que será enviado al modelo de lenguaje. El prompt combina:
1. La pregunta del usuario
2. Los fragmentos de documentos relevantes obtenidos en la fase de Retrieval.

Esto permite que el modelo responda no solo con su conocimiento entrenado, sino también con información actualizada de nuestra base vectorial.

In [10]:
# Preparamos la consulta del usuario en una sola línea (eliminamos saltos de línea)
user_query_clean  = user_query.replace("\n", " ")

# Construimos el prompt inicial del asistente
# - Se le indica al modelo que actúe como un asistente accesible y claro.
# - Incluimos la pregunta del usuario.
prompt = f""" Eres un asistente inteligente diseñado para brindar información
utilizando los documentos proporcionados como base. Tu tarea es responder las preguntas de forma completa, utilizando un lenguaje accesible y
amigable, dirigido a personas no técnicas.

PREGUNTAS: {user_query_clean}
"""

# Añadimos al prompt los documentos recuperados en la fase de Retrieval
for passage in retrieved_passages:
    passage_oneline = passage.replace("\n", " ")
    prompt += f"DOCUMENTO: {passage_oneline}\n"

# Mostramos el prompt final en formato Markdown
Markdown(prompt)

 Eres un asistente inteligente diseñado para brindar información
utilizando los documentos proporcionados como base. Tu tarea es responder las preguntas de forma completa, utilizando un lenguaje accesible y
amigable, dirigido a personas no técnicas.

PREGUNTAS: En qué año se registra el menor índice de pobreza en Perú?
DOCUMENTO: En el año 2020, la pobreza monetaria en Perú afectó al 30.1% de la población, lo que representó un incremento significativo de 9.9 puntos porcentuales respecto al año 2019. Esta cifra marcó un pico de pobreza, siendo el año 2020 el que alcanzó el nivel más alto desde el año 2010, debido a la fuerte crisis sanitaria provocada por la pandemia de COVID-19.
DOCUMENTO: En el 2015, el 21,77% de la población peruana vivía en situación de pobreza monetaria, lo que equivale a aproximadamente 6.782.000 personas, según el Instituto Nacional de Estadística e Informática (INEI). Este porcentaje representó una disminución de 1 punto porcentual respecto al año anterior, haciendo que 221 mil peruanos dejaran de ser pobres. La reducción de la pobreza se observó más en las regiones de Selva y Sierra, especialmente en la Selva urbana y Sierra rural.
DOCUMENTO: En el Perú, la pobreza monetaria en 2024 afectó al 27.6% de la población, lo que equivale a 9.395 millones de personas, según el INEI. Esta cifra representa una disminución de 1.4 puntos porcentuales respecto a 2023, mostrando una leve mejora, aunque los niveles aún están lejos de los registrados antes de la pandemia. La pobreza extrema, por su parte, se ubicó en 5.5% en 2024, también con una leve reducción.


En esta etapa enviamos el prompt enriquecido al modelo de lenguaje (Gemini) para obtener la respuesta final.

In [11]:
# Generamos la respuesta final del modelo usando el prompt enriquecido
final_answer  = client.models.generate_content(
    model="gemini-2.0-flash-lite",
    contents=prompt
    )

# Mostramos la respuesta en formato Markdown
Markdown(final_answer.text)

Según la información que tengo, el año con el menor índice de pobreza en Perú es el 2015. En ese año, el 21.77% de la población se encontraba en situación de pobreza.
